In [1]:
import torch
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [3]:
raw_data = np.loadtxt('diabetes.csv', delimiter=',', dtype=np.float32)
X = raw_data[:, :-1]
y = raw_data[:, [-1]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)

In [11]:
class DiabetesDataset(Dataset):
    def __init__(self, data, label):
        self.len = data.shape[0]
        self.x_data = torch.from_numpy(data)
        self.y_data = torch.from_numpy(label)
        
    def __getitem__(self,index):
        return self.x_data[index],self.y_data[index]
    
    def __len__(self):
        return self.len

train_dataset = DiabetesDataset(X_train, y_train)
train_loader = DataLoader(dataset = train_dataset, batch_size = 32, shuffle = True, num_workers = 0)

class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.linear1 = torch.nn.Linear(8,6)
        self.linear2 = torch.nn.Linear(6,4)
        self.linear3 = torch.nn.Linear(4,1)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        x = self.sigmoid(self.linear1(x))
        x = self.sigmoid(self.linear2(x))
        x = self.sigmoid(self.linear3(x))
        return x
    
model = Model()

criterion = torch.nn.BCELoss(reduction = 'mean')
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

def train(epoch):
    train_loss = 0
    count = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        y_pred = model(inputs)
        loss = criterion(y_pred, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        count = i
    
    if epoch % 2000 == 1999:
        print("train loss:",train_loss/count,end=',')
        
def test():
    with torch.no_grad():
        y_pred = model(X_test)
        y_pred_label = torch.where(y_pred >= 0.5,torch.tensor([1.0]),torch.tensor([0.0]))
        acc = torch.eq(y_pred_label, y_test).sum().item() / y_test.size(0)
        print("test acc:",acc)
        
if __name__ == '__main__':
    for epoch in range(50000):
        train(epoch)
        if epoch%2000 == 1999:
            test()

train loss: 0.4874582476913929,test acc: 0.7412280701754386
train loss: 0.4583042897284031,test acc: 0.7236842105263158
train loss: 0.46043875627219677,test acc: 0.7192982456140351
train loss: 0.46135210432112217,test acc: 0.7192982456140351
train loss: 0.4550736639648676,test acc: 0.7192982456140351
train loss: 0.4518254715949297,test acc: 0.7368421052631579
train loss: 0.4475599639117718,test acc: 0.75
train loss: 0.43342849612236023,test acc: 0.7587719298245614
train loss: 0.43642764165997505,test acc: 0.7719298245614035
train loss: 0.43799961172044277,test acc: 0.7719298245614035
train loss: 0.4351845681667328,test acc: 0.7631578947368421
train loss: 0.4281159583479166,test acc: 0.7587719298245614
train loss: 0.43511522095650434,test acc: 0.7675438596491229
train loss: 0.4265899397432804,test acc: 0.7675438596491229
train loss: 0.4221568237990141,test acc: 0.7631578947368421
train loss: 0.4220983274281025,test acc: 0.7543859649122807
train loss: 0.4189338758587837,test acc: 0.75
tr